In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
!pip install keras-tuner
import kerastuner

In [ ]:
!pip install wandb -qqq
import wandb

In [ ]:
!wandb login

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import kerastuner as kt
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
from wandb.keras import WandbCallback

In [ ]:
(trainX, trainY), (testX, testY) = tf.keras.datasets.cifar10.load_data()

In [ ]:
# one hot encode target values
trainY = to_categorical(trainY)
testY = to_categorical(testY)

In [ ]:
# convert from integers to floats
#train_norm = trainX.astype('float32')
#test_norm = testX.astype('float32')
# normalize to range 0-1
trainX = trainX / 255.0
testX = testX / 255.0

In [ ]:
# define cnn model
def build_model(hp):

  model = Sequential()

  hp_filters = hp.Int('filters', min_value = 32, max_value = 64, step = 32)
  model.add(Conv2D(filters=hp_filters, kernel_size=(3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
  
  model.add(BatchNormalization())
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.3))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.4))
  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))
  
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3])
  opt = Adam(learning_rate=hp_learning_rate)
  
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  
  return model

In [ ]:
class MyTuner(kt.Tuner):

    def run_trial(self, trial, trainX, trainY, batch_size, epochs, objective):

        hp = trial.hyperparameters
        objective_name_str = objective

        ## create the model with the current trial hyperparameters
        model = self.hypermodel.build(hp)
        
        ## Initiates new run for each trial on the dashboard of Weights & Biases
        run = wandb.init(project="WandBAndKerasTuner_RandomSearch", config=hp.values)

        ## WandbCallback() logs all the metric data such as
        ## loss, accuracy and etc on dashboard for visualization
        history = model.fit(trainX,
                  trainY,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_split=0.1,
                  callbacks=[WandbCallback()])  

        ## if val_accurcy used, use the val_accuracy of last epoch model which is fully trained
        val_acc = history.history['val_accuracy'][-1]  ## [-1] will give the last value in the list

        ## Send the objective data to the oracle for comparison of hyperparameters
        self.oracle.update_trial(trial.trial_id, {objective_name_str:val_acc})

        ## save the trial model
        self.save_model(trial.trial_id, model)
        
        ## ends the run on the Weights & Biases dashboard
        run.finish()

        

In [ ]:
## set the objective of tuning algorithm
objective = 'val_accuracy'
  
## instantiate the new Tuner with tuning algorithm and required parameters
tuner = MyTuner(
      oracle=kt.oracles.RandomSearch(
          objective=objective,
          max_trials=4),
      hypermodel=build_model,
      directory='./results_of_WandBwithKerasTuner7')

tuner.search_space_summary()

## initiates the hyperparameter tuning process
tuner.search(trainX, trainY, batch_size=32, epochs=5, objective=objective)

## get the best hyperparameters
best_hps = tuner.get_best_hyperparameters()[0]
print(best_hps.values)

## get the best
best_model = tuner.get_best_models()[0]
